# Xarray 101

Sources:
- [Xarray's Data Structures](https://tutorial.xarray.dev/fundamentals/01_datastructures.html)
- [Xarray in 45 minutes](https://tutorial.xarray.dev/overview/xarray-in-45-min.html)
- [Indexing and Selecting Data](https://tutorial.xarray.dev/fundamentals/02.1_indexing_Basic.html)
- [Grouped Computations](https://tutorial.xarray.dev/fundamentals/03.2_groupby_with_xarray.html)

## Overview: Why Xarray?

Multi-dimensional (a.k.a. N-dimensional, ND) arrays (sometimes called “tensors”)
are an essential part of computational science. They are encountered in a wide
range of fields, including physics, astronomy, geoscience, bioinformatics,
engineering, finance, and deep learning. In Python, [NumPy](https://numpy.org/)
provides the fundamental data structure and API for working with raw ND arrays.
However, real-world datasets are usually more than just raw numbers; they have
labels which encode information about how the array values map to locations in
space, time, etc.

Here is an example of how we might structure a dataset for a weather forecast:

<img src="https://docs.xarray.dev/en/stable/_images/dataset-diagram.png" align="center" width="80%">

You'll notice multiple data variables (temperature, precipitation), coordinate
variables (latitude, longitude), and dimensions (x, y, t). We'll cover how these
fit into Xarray's data structures below.

Xarray doesn’t just keep track of labels on arrays – it uses them to provide a
powerful and concise interface. For example:

- Apply operations over dimensions by name: `x.sum('time')`.

- Select values by label (or logical location) instead of integer location:
  `x.loc['2014-01-01']` or `x.sel(time='2014-01-01')`.

- Mathematical operations (e.g., `x - y`) vectorize across multiple dimensions
  (array broadcasting) based on dimension names, not shape.

- Easily use the split-apply-combine paradigm with groupby:
  `x.groupby('time.dayofyear').mean()`.

- Database-like alignment based on coordinate labels that smoothly handles
  missing values: `x, y = xr.align(x, y, join='outer')`.

- Keep track of arbitrary metadata in the form of a Python dictionary:
  `x.attrs`.

The N-dimensional nature of xarray’s data structures makes it suitable for
dealing with multi-dimensional scientific data, and its use of dimension names
instead of axis labels (`dim='time'` instead of `axis=0`) makes such arrays much
more manageable than the raw numpy ndarray: with xarray, you don’t need to keep
track of the order of an array’s dimensions or insert dummy dimensions of size 1
to align arrays (e.g., using np.newaxis).

The immediate payoff of using xarray is that you’ll write less code. The
long-term payoff is that you’ll understand what you were thinking when you come
back to look at it weeks or months later.

## Xarray's Data structures

Xarray provides two data structures: the `DataArray` and `Dataset`. The
`DataArray` class attaches dimension names, coordinates and attributes to
multi-dimensional arrays while `Dataset` combines multiple arrays.

Both classes are most commonly created by reading data.
To learn how to create a DataArray or Dataset manually, see the **Working with labeled data** tutorial.

Xarray has a few small real-world tutorial datasets hosted in the [pydata/xarray](https://github.com/pydata/xarray-data) Github repository.
We'll use the [xarray.tutorial.load_dataset](https://docs.xarray.dev/en/stable/generated/xarray.tutorial.open_dataset.html#xarray.tutorial.open_dataset) convenience function to download and open the `air_temperature` (National Centers for Environmental Prediction) Dataset by name.

In [ ]:
import numpy as np
import xarray as xr

In [ ]:
print("Numpy version:", np.__version__)
print("Xarray version:", xr.__version__)

### Dataset

`Dataset` objects are dictionary-like containers of DataArrays, mapping a variable name to each DataArray.

In [ ]:
ds = xr.tutorial.load_dataset("air_temperature")
ds

We can access "layers" of the Dataset (individual DataArrays) with dictionary syntax

In [ ]:
ds["air"]

We can save some typing by using the "attribute" or "dot" notation. This won't work for variable names that clash with built-in method names (for example, mean).

In [ ]:
ds.air

#### What is all this anyway? (String representations)

Xarray has two representation types: `"html"` (which is only available in
notebooks) and `"text"`. To choose between them, use the `display_style` option.

So far, our notebook has automatically displayed the `"html"` representation (which we will continue using).
The `"html"` representation is interactive, allowing you to collapse sections (left arrows) and
view attributes and values for each value (right hand sheet icon and data symbol).

In [ ]:
ds

In [ ]:
with xr.set_options(display_style="html"):
    display(ds)

The output consists of:

- a summary of all **Dimensions** of the `Dataset` `(lat: 25, time: 2920, lon: 53)`: this tells us that the first
  dimension is named `lat` and has a size of `25`, the second dimension is named
  `time` and has a size of `2920`, and the third dimension is named `lon` and has a size
  of `53`. Because we will access the dimensions by name, the order doesn't matter.
- an unordered list of **Coordinates** or dimensions with coordinates with one item
  per line. Each item has a name, one or more dimensions in parentheses, a dtype
  and a preview of the values. Also, if it is a dimension coordinate, it will be
  marked with a `*` or **bolded** like the example above.
- an alphabetically sorted list of **Data variables**, which are the actual data within the dataset. Similar to *Coordinates*, each data variable has a name, one or more dimensions in parentheses, a dtype and a preview of the values.
- an unordered list of **Indexes**, which lists the indexes associated with coordinates. This is a very new feature that is part of the [flexible indexes](https://docs.xarray.dev/en/stable/roadmap.html#flexible-indexes) work underway. Each item has a name and a class type of index used. This currently only shows up in the "html" representation as seen above.
- an unordered list of **Attributes**, or metadata

In [ ]:
with xr.set_options(display_style="text"):
    display(ds)

To understand each of the components better, we'll explore the "air" variable of our Dataset.

### DataArray

The `DataArray` class consists of an array (data) and its associated dimension names, labels, and attributes (metadata).

In [ ]:
da = ds["air"]
da

#### String representations

We can use the same two representations (`"html"`, which is only available in
notebooks, and `"text"`) to display our `DataArray`.

In [ ]:
with xr.set_options(display_style="html"):
    display(da)

In [ ]:
with xr.set_options(display_style="text"):
    display(da)

In the string representation of a `DataArray` (versus a `Dataset`), we also see:
- the `DataArray` name ('air')
- a preview of the array data (collapsible in the `"html"` representation)

We can also access the data array directly:

In [ ]:
ds.air.data # (or equivalently, `da.data`)

#### Named dimensions 

`.dims` are the named axes of your data. They may (dimension coordinates) or may not (dimensions without coordinates) have associated values. Names can be anything that fits into a Python `set` (i.e. calling `hash()` on it doesn't raise an error), but to be
useful they should be strings.

In this case we have 2 spatial dimensions (`latitude` and `longitude` are stored with shorthand names `lat` and `lon`) and one temporal dimension (`time`).

In [ ]:
ds.air.dims

#### Coordinates


`.coords` is a simple [dict-like](https://docs.python.org/3/glossary.html#term-mapping) [data container](https://docs.xarray.dev/en/stable/user-guide/data-structures.html#coordinates)
for mapping coordinate names to values. These values can be:
- another `DataArray` object
- a tuple of the form `(dims, data, attrs)` where `attrs` is optional. This is
  roughly equivalent to creating a new `DataArray` object with
  `DataArray(dims=dims, data=data, attrs=attrs)`
- a 1-dimensional `numpy` array (or anything that can be coerced to one using [`numpy.array`](https://numpy.org/doc/stable/reference/generated/numpy.array.html), such as a `list`) containing numbers, datetime objects, strings, etc. to label each point.

Here we see the actual timestamps and spatial positions of our air temperature data:

In [ ]:
ds.air.coords

:::{note}
The difference between the dimension labels (dimension coordinates) and normal
coordinates is that for now it only is possible to use indexing operations
(`sel`, `reindex`, etc.) with dimension coordinates. Also, while coordinates can
have arbitrary dimensions, dimension coordinates have to be one-dimensional.
:::

#### Attributes 

`.attrs` is a dictionary that can contain arbitrary Python objects (strings, lists, integers, dictionaries, etc.) containing information about your data. Your only
limitation is that some attributes may not be writeable to certain file formats.

In [ ]:
ds.air.attrs

### To Pandas and back

`DataArray` and `Dataset` objects are frequently created by converting from
other libraries such as [pandas](https://pandas.pydata.org/) or by reading from
data storage formats such as
[NetCDF](https://www.unidata.ucar.edu/software/netcdf/) or
[zarr](https://zarr.readthedocs.io/en/stable/).

To convert from / to `pandas`, we can use the
[`to_xarray`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_xarray.html)
methods on [pandas](https://zarr.readthedocs.io/en/stable/) objects or the
[`to_pandas`](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.to_pandas.html)
methods on `xarray` objects:

In [ ]:
import pandas as pd

In [ ]:
print("Pandas version:", pd.__version__)

In [ ]:
series = pd.Series(np.ones((10,)), index=list("abcdefghij"))
series

In [ ]:
arr = series.to_xarray()
arr

In [ ]:
arr.to_pandas()

We can also control what `pandas` object is used by calling `to_series` or
`to_dataframe`:

**`to_series`** will always convert `DataArray` objects to
`pandas.Series`, using a `MultiIndex` for higher dimensions

In [ ]:
ds.air.to_series()

**`to_dataframe`** will always convert `DataArray` or `Dataset`
objects to a `pandas.DataFrame`. Note that `DataArray` objects have to be named
for this.

In [ ]:
ds.air.to_dataframe()

Since columns in a DataFrame need to have the same index, they are *broadcasted*.

In [ ]:
ds.to_dataframe()

## Indexing and Selecting Data

Xarray offers extremely flexible indexing routines that combine the best features of NumPy and Pandas for data selection.

The most basic way to access elements of a `DataArray` object is to use Python’s `[]` syntax, such as `array[i, j]`, where `i` and `j` are both integers.

As xarray objects can store coordinates corresponding to each dimension of an array, label-based indexing is also possible (e.g. `.sel(latitude=0)`, similar to `pandas.DataFrame.loc`). In label-based indexing, the element position `i` is automatically looked-up from the coordinate values.

By leveraging the labeled dimensions and coordinates provided by Xarray, users can effortlessly access, subset, and manipulate data along multiple axes, enabling complex operations such as slicing, masking, and aggregating data based on specific criteria. 

This indexing and selection capability of Xarray not only enhances data exploration and analysis workflows but also promotes reproducibility and efficiency by providing a convenient interface for working with multi-dimensional data structures.

In total, xarray supports four different kinds of indexing, as described below and summarized in this table:

| Dimension lookup | Index lookup | `DataArray` syntax   |   `Dataset` syntax   |
| ---------------- | ------------ | ---------------------| ---------------------|
| Positional       | By integer   | `da[:,0]`            | *not available*      |
| Positional       | By label     | `da.loc[:,'IA']`     | *not available*      |
| By name          | By integer   | `da.isel(space=0)` or `da[dict(space=0)]`  | `ds.isel(space=0)` or  `ds[dict(space=0)]`  |
| By name          | By label     | `da.sel(space='IA')` or `da.loc[dict(space='IA')]` | `ds.sel(space='IA')` or `ds.loc[dict(space='IA')]` |

### Label-based indexing

Select data by coordinate labels. Xarray inherits its label-based indexing rules from pandas; this means great support for dates and times!

In [ ]:
# pull out data for all of 2013-May
ds.sel(time="2013-05")

In [ ]:
# slicing data from May to July 2013
ds.sel(time=slice("2013-05", "2013-07"))

In [ ]:
# can also get the whole year
ds.sel(time="2013")

In [ ]:
# demonstrate "nearest" indexing
ds.sel(lon=240.2, method="nearest")

In [ ]:
# "nearest indexing at multiple points"
ds.sel(lon=[240.125, 234], lat=[40.3, 50.3], method="nearest")

### Position-based indexing

This is similar to your usual numpy `array[0, 2, 3]` but with the power of named dimensions!

In [ ]:
ds.air.data[0, 2, 3]

In [ ]:
# pull out time index 0, lat index 2, and lon index 3
ds.air.isel(time=0, lat=2, lon=3)  #  much better than ds.air[0, 2, 3]

In [ ]:
# demonstrate slicing
ds.air.isel(lat=slice(10))

## Concepts for computation

Here is a motivating calculation where we subtract two DataArrays with data available at different locations in the (space, time) plane.

In [ ]:
arr1 = xr.DataArray(
    np.arange(12).reshape(3, 4),
    dims=("space", "time"),
    coords={"space": ["a", "b", "c"], "time": [0, 1, 2, 3]},
)
arr1

In [ ]:
arr2 = xr.DataArray(
    [0, 1],
    dims="space",
    coords={"space": ["b", "d"]},
)
arr2

:::{note}
`arr1` is 2D; while `arr2` is 1D along `space` and has values at two locations only.
:::

In [ ]:
arr1 - arr2

To understand this output, we must understand two fundamental concepts underlying computation with Xarray objects

1. Broadcasting: The objects need to have compatible shapes.
2. Alignment: The objects need to have values at the same coordinate labels

### Broadcasting: adjusting arrays to the same shape

**Broadcasting** allows an operator or a function to act on two or more arrays
to operate even if these arrays do not have the same shape. That said, not all
the dimensions can be subjected to broadcasting; they must meet certain rules.
The image below illustrates how an operation on arrays with
different coordinates will result in automatic broadcasting

![](https://tutorial.xarray.dev/_images/broadcasting_schematic.png)

Credit: Stephan Hoyer --
[xarray ECMWF Python workshop](https://docs.google.com/presentation/d/16CMY3g_OYr6fQplUZIDqVtG-SKZqsG8Ckwoj2oOqepU/)

Numpy's broadcasting rules, based on array shape, can sometimes be
difficult to understand and remember. Xarray does broadcasting by dimension name,
rather than array shape. This is a huge convenience.

Here are two 1D arrays

In [ ]:
array1 = xr.DataArray(
    np.arange(3),
    dims="space",
    coords={"space": ["a", "b", "c"]},
    name="array1",
)
array2 = xr.DataArray(
    np.arange(4),
    dims="time",
    coords={"time": [0, 1, 2, 3]},
    name="array2",
)
display(array1)
display(array2)

Let's subtract the two:

In [ ]:
array1 - array2

We see that the result is a 2D array. 

When subtracting, Xarray first realizes that `array1` is missing the dimension `time` and `array2` is missing the dimension `space`.  Xarray then broadcasts or "expands" both arrays to 2D with dimensions `space`, `time`. Here is an illustration:

![](https://tutorial.xarray.dev/_images/broadcasting_schematic.png)

#### Broadcasting in numpy

For contrast let us examine the pure numpy version  of this calculation. We use [.data](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.data.html) to extract the underlying numpy array object.


In [ ]:
array1.data - array2.data

To get this calculation to work, we need to insert new axes manually using [np.newaxis](https://numpy.org/doc/stable/reference/constants.html?highlight=newaxis).

In [ ]:
array1.data[:, np.newaxis] - array2.data[np.newaxis, :]


Because xarray knows about dimension names we avoid having to create unnecessary
size-1 dimensions using `np.newaxis` or `.reshape`. This is yet another example where the _metadata_ (dimension names) reduces the mental overhead associated with coding a calculation

For more, see the [Xarray documentation](https://docs.xarray.dev/en/stable/user-guide/computation.html#broadcasting-by-dimension-name) and the [numpy documentation](https://numpy.org/doc/stable/user/basics.broadcasting.html) on broadcasting.


### Alignment: putting data on the same grid


When combining two input arrays using an arithmetic operation, both arrays must first be converted to the same coordinate system. This is "alignment".

![](https://tutorial.xarray.dev/_images/alignment_schematic.png)


Here are two 2D DataArrays with different shapes.

In [ ]:
arr1 = xr.DataArray(
    np.arange(12).reshape(3, 4),
    dims=("space", "time"),
    coords={"space": ["a", "b", "c"], "time": [0, 1, 2, 3]},
)
arr1

In [ ]:
arr2 = xr.DataArray(
    np.arange(14).reshape(2, 7),
    dims=("space", "time"),
    coords={"space": ["b", "d"], "time": [-2, -1, 0, 1, 2, 3, 4]},
)
arr2

`arr1` and `arr2` have the same dimensions (space, time) but have values at different locations in the (space, time) plane  with some locations in common.

:::{note}
xarray assumes coordinate labels are in the same coordinate system such that space='b' in arr1 is the same as space='b' in arr2. For more sophisticated handling of coordinate systems see [rioxarray](https://corteva.github.io/rioxarray/stable/)
:::

We see that both arrays only have values in common at x="b" and y=[0, 1, 2, 3]. Before applying an arithmetic operation we must first modify each DataArray so that they have values at the same points. This is "alignment".

##### Controlling alignment

We can explicitly align objects using [xr.align](https://docs.xarray.dev/en/stable/generated/xarray.align.html). The key decision to make is how to decide which points must be kept. The other way to think of alignment is that objects must be converted to a common grid prior to any operation combining multiiple objects. This decision is controlled by the `"join"` keyword argument. Xarray provides 5 ways to convert the coordinate labels of multiple Datasets to a common grid. This [terminology](https://en.wikipedia.org/wiki/Join_(SQL)) originates in the database community.

1. `join="inner"` or reindex to the "intersection set" of coordinate labels
2. `join="outer"` or reindex to the "union set" of coordinate labels
3. `join="left"`  or reindex to the coordinate labels of the leftmost object
4. `join="right"` or reindex to the coordinate labels of the rightmost object
5. `join="exact"` checks for exact equality of coordinate labels before the operation.

First lets try an inner join. This is the default for arithmetic operations in Xarray. We see that the result has values for locations that `arr1` and `arr2` have in common: `x="b"` and `y=[0, 1, 2, 3]`. Here is an illustration

![](https://tutorial.xarray.dev/_images/alignment_schematic.png)

In [ ]:
a1_aligned, a2_aligned = xr.align(arr1, arr2, join="inner")
display(a1_aligned)
display(a2_aligned)

## High level computation

(`groupby`, `resample`, `rolling`, `coarsen`, `weighted`)

Xarray has some very useful high level objects that let you do common
computations:

1. `groupby` :
   [Bin data in to groups and reduce](https://docs.xarray.dev/en/stable/groupby.html)
1. `resample` :
   [Groupby specialized for time axes. Either downsample or upsample your data.](https://docs.xarray.dev/en/stable/user-guide/time-series.html#resampling-and-grouped-operations)
1. `rolling` :
   [Operate on rolling windows of your data e.g. running mean](https://docs.xarray.dev/en/stable/user-guide/computation.html#rolling-window-operations)
1. `coarsen` :
   [Downsample your data](https://docs.xarray.dev/en/stable/user-guide/computation.html#coarsen-large-arrays)
1. `weighted` :
   [Weight your data before reducing](https://docs.xarray.dev/en/stable/user-guide/computation.html#weighted-array-reductions)


Below we quickly demonstrate these patterns. See the user guide links above and [the tutorial](https://tutorial.xarray.dev/intermediate/01-high-level-computation-patterns.html) for more.

### `groupby`

Xarray copies Pandas’ very useful groupby functionality, enabling the “split / apply / combine” workflow on xarray DataArrays and Datasets.

In [ ]:
# seasonal groups
ds.groupby("time.season")

In [ ]:
# make a seasonal mean
seasonal_mean = ds.groupby("time.season").mean()
seasonal_mean

The seasons are out of order (they are alphabetically sorted). This is a common
annoyance. The solution is to use `.sel` to change the order of labels

In [ ]:
seasonal_mean = seasonal_mean.sel(season=["DJF", "MAM", "JJA", "SON"])
seasonal_mean

### `resample`

Resampling means changing the time frequency of data, usually reducing to a coarser frequency: e.g. converting daily frequency data to monthly frequency data using `mean` to reduce the values. This operation can be thought of as a groupby operation where each group is a single month of data. Resampling can be applied only to time-index dimensions.

In [ ]:
# resample to monthly frequency
ds.resample(time="M").mean()

### `weighted`

Xarray supports [weighted array reductions](https://docs.xarray.dev/en/stable/user-guide/computation.html#weighted-array-reductions)

`DataArray` and `Dataset` objects include `DataArray.weighted()` and `Dataset.weighted()` array reduction methods. They currently support weighted `sum`, `mean`, `std`, `var` and `quantile`.

In [ ]:
coords = dict(month=("month", [1, 2, 3]))

In [ ]:
prec = xr.DataArray([1.1, 1.0, 0.9], dims=("month",), coords=coords)

In [ ]:
weights = xr.DataArray([31, 28, 31], dims=("month",), coords=coords)

In [ ]:
# Created weighted object
weighted_prec = prec.weighted(weights)
weighted_prec

In [ ]:
# Calculate the weighted sum
weighted_prec.sum()

In [ ]:
# Calculate the weighted mean
weighted_prec.mean(dim="month")

In [ ]:
# Calculate the weighted quantile
weighted_prec.quantile(q=0.5, dim="month")

:::{note}
weights must be a `DataArray` and cannot contain missing values. Missing values can be replaced manually by `weights.fillna(0)`.
:::

## Visualization

Xarray lets you easily visualize datasets easily by default and integrates really well with the [Holoviz](https://holoviz.org/) ecosystem.

Plot the air temperature seasonal mean from above `groupby`

In [ ]:
seasonal_mean.air.plot(col="season", col_wrap=2);

In [ ]:
# contours
seasonal_mean.air.plot.contour(col="season", levels=20, add_colorbar=True);

In [ ]:
# line plots as well
seasonal_mean.air.mean("lon").plot.line(hue="season", y="lat");

For more see the [user guide](https://docs.xarray.dev/en/stable/plotting.html), the [gallery](https://docs.xarray.dev/en/stable/examples/visualization_gallery.html), and [the tutorial material](https://tutorial.xarray.dev/fundamentals/04.0_plotting.html).

## Reading and writing files

Xarray supports many disk formats. Below is a small example using netCDF. For
more see the [documentation](https://docs.xarray.dev/en/stable/user-guide/io.html)

In [ ]:
ds.to_netcdf("my-example-dataset.nc")

```{note}
To avoid the `SerializationWarning` you can assign a _FillValue for any NaNs in 'air' array by adding the keyword argument encoding=dict(air={_FillValue=-9999})
```

In [ ]:
# read from disk
fromdisk = xr.open_dataset("my-example-dataset.nc")
fromdisk

In [ ]:
# check that the two are identical
ds.identical(fromdisk)

:::{tip}
A common use case to read datasets that are a collection of many netCDF
files. See the [documentation](https://docs.xarray.dev/en/stable/user-guide/io.html#reading-multi-file-datasets) for how
to handle that.

Finally to read other file formats, you might find yourself reading in the data using a different library and then creating a DataArray([docs](https://docs.xarray.dev/en/stable/user-guide/data-structures.html#creating-a-dataarray), [tutorial](https://tutorial.xarray.dev/fundamentals/01.1_creating_data_structures.html)) from scratch. For example, you might use `h5py` to open an HDF5 file and then create a Dataset from that.
For MATLAB files you might use `scipy.io.loadmat` or `h5py` depending on the version of MATLAB file you're opening and then construct a Dataset.
:::

## What's Next

1. Read the [tutorial](https://tutorial.xarray.dev) material and [user guide](https://docs.xarray.dev/en/stable/user-guide/index.html)
1. See the description of [common terms](https://docs.xarray.dev/en/stable/terminology.html) used in the xarray documentation
1. Answers to common questions on "how to do X" with Xarray are [here](https://docs.xarray.dev/en/stable/howdoi.html)
1. Ryan Abernathey has a book on data analysis with a [chapter on Xarray](https://earth-env-data-science.github.io/lectures/xarray/xarray_intro.html)
1. [Project Pythia](https://projectpythia.org/) has [foundational](https://foundations.projectpythia.org/landing-page.html) and more [advanced](https://cookbooks.projectpythia.org/) material on Xarray. Pythia also aggregates other [Python learning resources](https://projectpythia.org/resource-gallery.html).
1. The [Xarray Github Discussions](https://github.com/pydata/xarray/discussions) and [Pangeo Discourse](https://discourse.pangeo.io/) are good places to ask questions.
1. Tell your friends! Tweet!